<a href="https://colab.research.google.com/github/Haripriya-Mahajan/AI-Model-Comparison-Tool/blob/main/AIModelComparisonTool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# =========================
# LLM Research Project (NO OpenAI, FIXED)
# =========================

# 1️⃣ Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

!pip install PyPDF2 pandas requests

import os
import pickle
import PyPDF2
import pandas as pd
import re

# -------------------------
# 2️⃣ Paths
# -------------------------
CACHE_DIR = "/content/drive/MyDrive/LLM_Project_Cache"
os.makedirs(CACHE_DIR, exist_ok=True)

PDF_CACHE_PATH = os.path.join(CACHE_DIR, "pdf_texts.pkl")
CSV_CACHE_PATH = os.path.join(CACHE_DIR, "benchmark_data.pkl")

# -------------------------
# 3️⃣ PDF text extraction
# -------------------------
def extract_text_from_pdf(pdf_path):
    text = ""
    try:
        with open(pdf_path, "rb") as f:
            reader = PyPDF2.PdfReader(f, strict=False)
            for page in reader.pages:
                page_text = page.extract_text()
                if page_text:
                    text += page_text + "\n"
    except Exception as e:
        print(f"⚠️ Skipping corrupted PDF: {pdf_path}")
        print(f"   Reason: {e}")

    if not text.strip():
        print(f"⚠️ No extractable text found in {pdf_path}")

    return text

# -------------------------
# 4️⃣ Load or upload PDFs
# -------------------------
if os.path.exists(PDF_CACHE_PATH):
    with open(PDF_CACHE_PATH, "rb") as f:
        pdf_texts = pickle.load(f)
    print("✅ Loaded cached PDF texts from Drive")
else:
    from google.colab import files
    print("📤 Upload ONLY PDF files now")
    uploaded = files.upload()

    pdf_texts = []

    for file_name in uploaded:
        if file_name.lower().endswith(".pdf"):
            text = extract_text_from_pdf(file_name)
            pdf_texts.append(text)
            print(f"Extracted {len(text)} characters from {file_name}")

    with open(PDF_CACHE_PATH, "wb") as f:
        pickle.dump(pdf_texts, f)

    print("✅ PDF texts cached to Drive")

# -------------------------
# 5️⃣ Load or upload CSV (ONCE)
# -------------------------
if os.path.exists(CSV_CACHE_PATH):
    with open(CSV_CACHE_PATH, "rb") as f:
        benchmark_df = pickle.load(f)
    print("✅ Loaded cached benchmark CSV from Drive")
else:
    from google.colab import files
    print("📤 Upload ONLY the benchmark CSV now")
    uploaded = files.upload()

    csv_file = [f for f in uploaded if f.lower().endswith(".csv")][0]
    benchmark_df = pd.read_csv(csv_file)

    benchmark_df = benchmark_df.drop(
        columns=[c for c in ['Benchmark paper', 'Code repository', 'Dataset '] if c in benchmark_df.columns],
        errors="ignore"
    )

    with open(CSV_CACHE_PATH, "wb") as f:
        pickle.dump(benchmark_df, f)

    print("✅ CSV cached to Drive")

# -------------------------
# 6️⃣ Query processing (NO LLM)
# -------------------------
def answer_query(query, pdf_texts, df):
    query = query.lower()
    keywords = re.findall(r"\w+", query)

    # --- Search PDFs ---
    pdf_hits = []
    for text in pdf_texts:
        score = sum(text.lower().count(k) for k in keywords)
        if score > 0:
            pdf_hits.append((score, text[:500]))

    pdf_hits.sort(reverse=True)

    # --- Search CSV ---
    csv_hits = df[df.apply(
        lambda row: any(k in str(row).lower() for k in keywords),
        axis=1
    )]

    # --- Build answer ---
    answer = "\n📌 RESULTS BASED ON DATA\n\n"

    if not csv_hits.empty:
        answer += "🔹 Relevant Models from Benchmarks:\n"
        answer += csv_hits.head(5).to_string(index=False)
        answer += "\n\n"

    if pdf_hits:
        answer += "🔹 Evidence from Research Papers:\n"
        answer += pdf_hits[0][1]
        answer += "\n"

    if not pdf_hits and csv_hits.empty:
        answer += "No relevant information found."

    return answer

# -------------------------
# 7️⃣ User query
# -------------------------
question = input("Enter your research question: ")
result = answer_query(question, pdf_texts, benchmark_df)

print("\n" + "="*50)
print(result)
print("="*50)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Loaded cached PDF texts from Drive
✅ Loaded cached benchmark CSV from Drive
Enter your research question: Which LLM performs best for question answering tasks?


📌 RESULTS BASED ON DATA

🔹 Relevant Models from Benchmarks:
           Name                              Type                                                                                                                                                                           Description Number of examples                License
      MultiLoKo language & reasoning,multilingual                                                                                                         A new benchmark for evaluating multilinguality in LLMs covering 31 languages.              15500            MIT License
FACTS Grounding                            safety                                                  